## I. Збір та анотування даних  
### Опрацюйте один із архівів Wiktionary і витягніть усі синонімні ряди для будь-якої мови. Будь ласка, не обирайте англійську, українську чи російську мови. 
Был выбран шведский язык.  
Пользовался последним дампом - `svwiktionary-latest-pages-articles.xml`  
Скрипт - `parsing.py`  
Результат выполнения скрипта:

In [15]:
import pandas as pd

In [16]:
!ls

homework03.ipynb
official-2014.combined-withalt.m2
parsing.py
svwiktionary-latest-pages-articles-multistream.xml
svwiktionary-latest-pages-articles-multistream.xml.bz2
syns.pkl


In [20]:
unpickled_df = pd.read_pickle("./syns.pkl")
unpickled_df

,word,syn
0,besserwisser [know-it-all],"[bättrevetare, viktigpetter, messerschmitt]"
1,Sverige [Sweden],"[landet lagom, moder Svea, Svea rike, Konungar..."
2,barr [barre],"[balettstång, stång]"
3,samlag [intercourse],"[sex, knull, älskog]"
4,knulla [fuck],"[banga, borra, bulta, doppa, dra över, dunka, ..."
...,...,...
22840,namnkristen,"[skenkristen, mun kristen]"
22841,omnipotente,[todo poderoso]
22842,förlängning,"[sudden, övertid, sudden death, sudden death]"
22843,tillägstid,"[sudden, övertid, sudden death, sudden death]"


### Визначте рівень згоди між анотувальниками (або inter-annotator agreement) у корпусі NUCLE Error Corpus. 

In [26]:
from itertools import combinations

tags = """Any Vt3 Vm V0 Vform SVA ArtOrDet Nn Npos Pform Pref Prep Wci Wa Wform Wtone Srun Smod Spar Sfrag Ssub WOinc WOadv Trans Mec Rloc Cit Others Um""".split(" ")


import pathlib
curr_path = pathlib.Path().parent.absolute()
path = curr_path / 'official-2014.combined-withalt.m2'

lines = []
with open(path) as file:
    lines = file.readlines()

In [27]:
comb = list(combinations(range(5), 2))

data_for_tag = {}
pairvise_agreement = {}

for t in tags:
    data_for_tag[t]={}

for c in comb:
    pairvise_agreement[c] = []

#prepare data 
for t in tags:
    for c in comb:
        data_for_tag[t][c]=[]

examples = []
curr = []
for line in lines:
    if line!="\n":
        curr.append(line)
    else:
        examples.append(curr)
        curr = [] 

def create_dict():
    d = {}
    for tag in tags: 
        d[tag] = []
    return d

def get_anotator(s):
    return int(s.strip()[-1])

def get_anotation_string(s, sentense_number, tag):
    i = s.find("REQUIRED")
    s = s.strip()[2:i]
    token_range = ""
    count = 0
    for x in s:
        if x=="|":
            break
        count += 1
        token_range += x
    s = s[count+3:]
    count = 0 
    anotation_tag=""
    for x in s:
        if x=="|":
            break
        anotation_tag += x
        count += 1
    s = s[count+3:]
    if tag!="Any":
        if anotation_tag!=tag:
            return ""
    correction=""
    count=0
    for x in s:
        if x=="|":
            break
        correction+=x
        count+=1
    return " ".join([str(sentense_number), token_range, correction])

def get_anotation_range(s):
    range_string = ""
    for x in s[2:]:
        if x=="|":
            break
        range_string+=x
    return tuple(map(int, range_string.split(' '))) 

def get_fix(s):
    return get_anotation_string(s, 0, "Any").split(' ')[-1]

def dist(r):
    return r[1]-r[0]
    
def process_block(l, index):
    anotations = l[1:]
    if len(anotations) > 1:
        anotation_tags = create_dict()
        t = [""]*5
        ranges = [[] for _ in range(5)]
        fixes = [[] for _ in range(5)]
        curr_anotator = -1
        for x in anotations:
            an = get_anotator(x)
            if an != curr_anotator:
                if curr_anotator!=-1:
                    t[curr_anotator] = anotation_tags
                    anotation_tags = create_dict()
                curr_anotator = an
            ranges[an].append(get_anotation_range(x))
            fixes[an].append(get_fix(x))
            for tag in tags:
                anotation_tags[tag].append(get_anotation_string(x, index, tag))

        t[curr_anotator] = anotation_tags
        #append processed lines to data 
        indexes = [i for i, j in enumerate(t) if j!='']
        if len(indexes)>1:
            #process in general 
            for idx in combinations(indexes, 2):
                i, j = idx
                ri=ranges[i]
                rj=ranges[j]
                fi=fixes[i]
                fj=fixes[j]
                #go through ranges (check if range needs to be expanded) for left term
                expanded_ri = []
                expanded_fi = []
                for i, r in enumerate(ri):
                    if dist(r)==2:
                        fix = fi[i]
                        if len(fix.split())==1: #we're dealing with 2->1 word fix, meaning second word would be deleted
                            expanded_ri.append((r[0],r[0]+1))
                            expanded_fi.append(fi[i])
                            expanded_ri.append((r[1],r[1]+1))
                            expanded_fi.append("")
                    else:
                        expanded_ri.append(r)
                        expanded_fi.append(fi[i])
                #go through ranges (check if range needs to be expanded) for right term
                expanded_rj = []
                expanded_fj = []
                for i, r in enumerate(rj):
                    if dist(r)==2:
                        fix = fj[i]
                        if len(fix.split())==1: #we're dealing with 2->1 word fix, meaning second word would be deleted
                            expanded_rj.append((r[0],r[0]+1))
                            expanded_fj.append(fj[i])
                            expanded_rj.append((r[1],r[1]+1))
                            expanded_fj.append("")
                    else:
                        expanded_rj.append(r)
                        expanded_fj.append(fj[i])
                g = [expanded_rj.count(el) for el in expanded_ri]
                if len(g)>0:
                    c = sum(g)*2/(len(expanded_ri)+len(expanded_rj))
                    pairvise_agreement[idx].append(c)

            #process for each tag
            for tag in tags:
                for idx in combinations(indexes, 2):
                    d1 = t[idx[0]][tag]
                    d2 = t[idx[1]][tag]
                    g = [d2.count(el) for el in d1 if el!='']
                    if len(g)!=0:
                        c = sum(g)*2/(len(d1)+len(d2))
                        data_for_tag[tag][idx].append(c)


for i, e in enumerate(examples):
    process_block(e,i)


#calculating result 
print("Agreement by PAIR:")
all=[]
for c in comb:
    d = pairvise_agreement[c]
    all.extend(d)
    if len(d)!=0: 
        combined_score = sum(d)/len(d)
        print("Pair: {} Score: {}".format(c, combined_score))

print("General agreement:")
print(sum(all)/len(all))

print("Agreement by TAG:")
for tag in tags:
    data = data_for_tag[tag]
    combined_score = 0
    count = 0
    for k in data:
        d = data[k]
        if len(d)!=0: # Means this annotators annotated same sentences with this tag
            combined_score += sum(d)/len(d)
            count += 1

    res = combined_score/count if count!=0 else 0
    print("For tag: {} score is: {}".format(tag, res))

Agreement by PAIR
Pair: (0, 1) Score: 0.33398914582172073
Pair: (0, 2) Score: 0.4069804408428973
Pair: (0, 3) Score: 0.5096655053092518
Pair: (0, 4) Score: 0.7188692571045512
Pair: (1, 2) Score: 0.49448138169155953
Pair: (1, 3) Score: 0.5614817226249059
Pair: (1, 4) Score: 0.6329485329485329
Pair: (2, 3) Score: 0.5457730206877925
Pair: (2, 4) Score: 0.5416666666666667
Pair: (3, 4) Score: 0.5035714285714286
General agreement:
0.3952901650568998
Agreement by TAG
For tag: Any score is: 0.4339096067012667
For tag: Vt3 score is: 0
For tag: Vm score is: 0.059583316422409
For tag: V0 score is: 0.08932971231964114
For tag: Vform score is: 0.1371039023483001
For tag: SVA score is: 0.10700282848347095
For tag: ArtOrDet score is: 0.1578472245092778
For tag: Nn score is: 0.16807668845741963
For tag: Npos score is: 0.05250616612549034
For tag: Pform score is: 0.0683225705034294
For tag: Pref score is: 0.08492123276948227
For tag: Prep score is: 0.15783307874541927
For tag: Wci score is: 0.140629583

Conclusions:
Multiple,  different  corrections  are  of-ten acceptable. In general considering all tags score showed moderate agreement between peers. 

Higher scores are between annotators who have less common sentences. The lowest score is between two annotators who have the most sentences processed together. 

Tags that have 0 agreement are the ones that werent used by any two annotators on a same sample 
